In [1]:
import numpy as np
import pandas as pd
import math
import plotly.express as px
import statistics
import itertools as it
from functions import mms_enspectra_simulation_functions
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as sm
from plotly import subplots
import random
from functions import data_preprocess_functions
import glob
import ipyplot

#import pytplot

In [2]:
regr = LinearRegression()
goodness_of_fit_threshold = 0.6
model = "t89"
radius_earth = 6378.14 #km

data_filename = 'data/fulldata_20160101_to_20171231.csv'
dispersion_filename = 'output/dispersion_list.csv'
dispersion_m_filename = 'data/dispersion list - mms.csv'
dispersion_merged_filename = 'output/merged_list.csv'

In [3]:
data = data_preprocess_functions.preprocess_data(pd.read_csv(data_filename))
data = data.loc[data['N_DISPERSION_PARA'].notnull() | data['N_DISPERSION_ANTI'].notnull(),:]
data['fit_error'] = data['DIS_FITTING_SIGMA_PARA']*radius_earth

dispersion_list = data_preprocess_functions.extract_dispersions(data)
dispersion_list_m = pd.read_csv(dispersion_m_filename)
dispersion_list_m_merged = pd.read_csv(dispersion_merged_filename)

dispersion_tplot_list = glob.glob("idl_plots/dispersion_day/*.png")
dispersion_fitting_plot_list = glob.glob("idl_plots/dispersion/*.png")

ensize = len(mms_enspectra_simulation_functions.MMS_ENERGY_BIN)


In [4]:
# grid distant samples:
# dist_set = [5, 8.69, 10, 11.57, 16.85, 20, 35, 50, 65]

dist_set = round(dispersion_list_m_merged.loc[~np.isnan(dispersion_list_merged['model_field_line_length_idl'])
                                            , 'model_field_line_length_idl'].sort_values(),2)

NameError: name 'dispersion_list_merged' is not defined

In [ ]:
dsize = len(dist_set)

In [ ]:
#vsize = 682
#vel_set = np.arange(vsize)+10

random.seed(42)
vel_set = []
for i in range(1000):
    vel_set.append(random.betavariate(2, 3)*687 + 10)
vel_set.sort()
px.histogram(vel_set)

In [ ]:
result = pd.DataFrame(np.array(np.meshgrid(vel_set, dist_set)).T.reshape(-1,2),columns=["velocity","distance_Re"])

result['distance'] = result['distance_Re'].apply(mms_enspectra_simulation_functions.convert_Re_to_km)
result['energy'] = result['velocity'].apply(mms_enspectra_simulation_functions.calcualte_energy_from_velocity)
result['time'] = result['distance'] / result['velocity'] / 60.
result['energy_binned'] = mms_enspectra_simulation_functions.find_energy_bin_df(result['energy'])

In [ ]:
fig = px.line(result, x="time", y="energy_binned", color='distance_Re', log_y=True)
fig.update_xaxes(dtick=5)
fig.show()

In [ ]:
target_distances = [8.61]
for target_distance in target_distances:
    index = result['distance_Re'] == target_distance
    
    fig = px.scatter(result.loc[index,:], x="time", y="energy_binned", log_y=True
                     , title= str(target_distance)+' Re')
    fig.update_xaxes(dtick=5)
    fig.show()

In [ ]:
toplot = dispersion_list_m_merged.loc[dispersion_list_m_merged['Full list Index'].notnull(),:]
for i in [0]:
    display_array = []
    i_toplot = toplot.loc[i,:]
    match_date = lambda x: (i_toplot['date'][0:4]+i_toplot['date'][5:7]+i_toplot['date'][8:10]) in x
    match_list1 = list(map(match_date, dispersion_tplot_list))
    if match_list1.count(True) > 0:
        index1 = match_list1.index(True)
        i_tplot1 = dispersion_tplot_list[index1]
        display_array.append(i_tplot1)

    match_list2 = list(map(match_date, dispersion_fitting_plot_list))
    if match_list2.count(True) > 0:
        index2 = match_list2.index(True)
        i_tplot2 = dispersion_fitting_plot_list[index2]
        display_array.append(i_tplot2)

    ipyplot.plot_images(display_array, img_width=370)

# display(Image(url = display_array[0]))

In [ ]:
target_distances = [8.61]
avg_time = 5. 

for target_distance in target_distances:
    energy_spectra = mms_enspectra_simulation_functions.calculate_energy_spectra(result, target_distance, avg_time)

    fig = px.scatter(energy_spectra, x="time", y="energy_binned", color = 'flux', log_y=True
                     , title = 'distance: ' + str(target_distance) + ' Re, ' 
                     + '   average time: ' + str(avg_time) + ' min')
    fig.update_xaxes(dtick=avg_time)
    fig.show()


In [ ]:
target_distances = [8.61]
avg_time = 5. 

for target_distance in target_distances:
    energy_spectra = mms_enspectra_simulation_functions.calculate_energy_spectra(result, target_distance, avg_time)
    beam = mms_enspectra_simulation_functions.identify_beam(energy_spectra)
    fig = px.scatter(energy_spectra, x="time", y="energy_binned", color = 'flux', log_y=True
                     , title = 'distance: ' + str(target_distance) + ' Re, ' 
                     + '   average time: ' + str(avg_time) + ' min')

    reference_line = go.Scatter(x = list(beam['time']), y = list(beam['energy_binned'])
                    , mode = "lines", line = go.scatter.Line(color="gray"), showlegend=False)

    fig.add_trace(reference_line)

    fig.update_xaxes(dtick=avg_time)
    fig.show()

In [ ]:
target_distances = [8.61]
avg_time = 5. 

for target_distance in target_distances:
    energy_spectra = mms_enspectra_simulation_functions.calculate_energy_spectra(result, target_distance, avg_time)
    beam = mms_enspectra_simulation_functions.identify_beam(energy_spectra)
    dispersion = mms_enspectra_simulation_functions.identify_dispersion(beam)

    fig = px.scatter(energy_spectra, x="time", y="energy_binned", color = 'flux', log_y=True
                     , title = 'distance: ' + str(target_distance) + ' Re, ' 
                     + '   average time: ' + str(avg_time) + ' min')

    reference_line1 = go.Scatter(x = list(beam['time']), y = list(beam['energy_binned'])
                            , mode = "lines", line = go.scatter.Line(color="gray"), showlegend=False)

    reference_line2 = go.Scatter(x = list(dispersion['time']), y = list(dispersion['energy_binned'])
                            , mode = "lines", line = go.scatter.Line(color="red"), showlegend=False)    

    fig.add_trace(reference_line1)
    fig.add_trace(reference_line2)

    fig.update_xaxes(dtick=avg_time)
    fig.show()
            

In [ ]:
data.columns

In [ ]:
dispersion_list_m_merged 

In [ ]:
target_distances = [8.61]
avg_time = 5. 

for target_distance in target_distances:
    energy_spectra = mms_enspectra_simulation_functions.calculate_energy_spectra(result, target_distance, avg_time)
    beam = mms_enspectra_simulation_functions.identify_beam(energy_spectra)
    dispersion = mms_enspectra_simulation_functions.identify_dispersion(beam)
    mydata = data[]
    
    
    fig = px.scatter(energy_spectra, x="time", y="energy_binned", color = 'flux', log_y=True
                     , title = 'distance: ' + str(target_distance) + ' Re, ' 
                     + '   average time: ' + str(avg_time) + ' min')

    reference_line1 = go.Scatter(x = list(beam['time']), y = list(beam['energy_binned'])
                            , mode = "lines", line = go.scatter.Line(color="gray"), showlegend=False)

    reference_line2 = go.Scatter(x = list(dispersion['time']), y = list(dispersion['energy_binned'])
                            , mode = "lines", line = go.scatter.Line(color="red"), showlegend=False)    

    fig.add_trace(reference_line1)
    fig.add_trace(reference_line2)

    fig.update_xaxes(dtick=avg_time)
    fig.show()
            

In [ ]:
toplot = dispersion_list_m_merged.loc[dispersion_list_m_merged['Full list Index'].notnull(),:]
for i in [0]:
    display_array = []
    i_toplot = toplot.loc[i,:]
    match_date = lambda x: (i_toplot['date'][0:4]+i_toplot['date'][5:7]+i_toplot['date'][8:10]) in x
    match_list1 = list(map(match_date, dispersion_tplot_list))
    if match_list1.count(True) > 0:
        index1 = match_list1.index(True)
        i_tplot1 = dispersion_tplot_list[index1]
        display_array.append(i_tplot1)

    match_list2 = list(map(match_date, dispersion_fitting_plot_list))
    if match_list2.count(True) > 0:
        index2 = match_list2.index(True)
        i_tplot2 = dispersion_fitting_plot_list[index2]
        display_array.append(i_tplot2)

    ipyplot.plot_images(display_array, img_width=370)

# display(Image(url = display_array[0]))

In [ ]:
target_distances = dist_set
avg_time = 5. 
dispersion_list = pd.DataFrame()

for target_distance in target_distances:
    energy_spectra = mms_enspectra_simulation_functions.calculate_energy_spectra(result, target_distance, avg_time)
    beam = mms_enspectra_simulation_functions.identify_beam(energy_spectra)
    dispersion = mms_enspectra_simulation_functions.identify_dispersion(beam)
    
    fig1 = px.scatter(energy_spectra, x="time", y="energy_binned", color = 'flux', log_y=True
                     , title = 'distance: ' + str(target_distance) + ' Re, ' 
                     + '   average time: ' + str(avg_time) + ' min')
    
    reference_line1 = go.Scatter(x = list(beam['time']), y = list(beam['energy_binned'])
                            , mode = "lines", line = go.scatter.Line(color="gray"), showlegend=False)

    reference_line2 = go.Scatter(x = list(dispersion['time']), y = list(dispersion['energy_binned'])
                            , mode = "lines", line = go.scatter.Line(color="red"), showlegend=False)    

    fig1.add_trace(reference_line1)
    fig1.add_trace(reference_line2)
    
    fig1.update_xaxes(dtick=avg_time)
    fig1.show()
       
    for idispersion in (np.unique(dispersion.loc[~np.isnan(dispersion['ndisperison']),'ndisperison'])):
        index = dispersion['ndisperison'] == idispersion

        x = np.array(dispersion.loc[index, 'time']*60).reshape(-1)
        y = np.array(dispersion.loc[index, 'inverse_v']).reshape(-1)
        ws = pd.DataFrame({'x': x, 'y': y})
        y_err = dispersion.loc[index, 'd_inverse_v']
        weights = pd.Series(y_err)

        wls_fit = sm.wls('y ~ x', data=ws, weights=1/weights).fit()
        ols_fit = sm.ols('y ~ x', data=ws).fit()

        dispersion.loc[index, 'estimated_distance'] = 1/wls_fit.params.x/6731.
        dispersion.loc[index, 'distance'] = target_distance

        fig2 = px.scatter(x = x, y = y, error_y= y_err
                         ,title = 'distance: ' + str(target_distance) + ' Re, ' + '   average time: ' 
                         + str(avg_time) + ' min, estimated distance: ' + str(round(1/wls_fit.params.x/6731., 2))
                          + ", r adjusted: "+str(round(ols_fit.rsquared_adj,2))
                         , labels=dict(x="time", y='1 / v'))
        
        reference_line1 = go.Scatter(x = x, y = ols_fit.predict(),  mode = "lines"
                                     , line = go.scatter.Line(color="grey"), showlegend=True,name='linear fit')
        reference_line2 = go.Scatter(x = x, y = wls_fit.predict(), mode = "lines"
                                     , line = go.scatter.Line(color="red"), showlegend=True, name='weighted linear fit')

        fig2.add_trace(reference_line1)
        fig2.add_trace(reference_line2)
    
        fig2.show()
    dispersion_list = dispersion_list.append(dispersion)

In [ ]:
index = ~dispersion_list['estimated_distance'].apply(np.isnan)
dispersion_summary = dispersion_list.loc[index, ['estimated_distance', 'distance']].drop_duplicates()
fig = px.scatter(dispersion_summary, x='distance',y='estimated_distance')
reference_line = go.Scatter(x=[0, 70], y=[0, 70], mode="lines", line=go.scatter.Line(color="gray"), showlegend=False)
fig.add_trace(reference_line)

In [ ]:
#new_energy_spectra = energy_spectra.pivot(index='energy_binned', columns='time', values='flux')

#fig = px.imshow(np.array(new_energy_spectra)
#                , x = list(np.unique(new_energy_spectra['time']))
#                , y = list(np.unique(new_energy_spectra['energy_binned']))
#                , title = 'distance: '+str(target_distance)+' Re, '+'   average time: '+str(avg_time)+' min')
#fig.update_xaxes(dtick=avg_time)
#fig.show()

#pytplot.store_data("enspectra"
#                   , data={  'x': np.unique(energy_spectra['time'])
#                           , 'y': np.unique(energy_spectra['flux']) 
#                           , 'v': energy_spectra['energy_binned']})